In [31]:
%autoreload 2

import recommender_utils
import numpy as np
import scipy.sparse as sp
import pandas as pd

In [20]:
movie_df, genre_headers, num_genres = recommender_utils.get_movies_data(filepath='data/movies.csv', separator=r',', movies_columns_to_drop=['genres'])

data_train = recommender_utils.get_ratings_data(filepath='data_small/train.csv', separator=r',', dtypes=recommender_utils.dtypes)
data_val = recommender_utils.get_ratings_data(filepath='data_small/validate.csv', separator=r',', dtypes=recommender_utils.dtypes)
data_test = recommender_utils.get_ratings_data(filepath='data_small/test.csv', separator=r',', dtypes=recommender_utils.dtypes)

data_array_train = np.array(data_train.values.tolist())
data_array_val = np.array(data_val.values.tolist())
data_array_test = np.array(data_test.values.tolist())

# add features for whole dataset TODO
train_adjacency_mx, train_labels, train_user_idx, train_item_idx, train_item_dict = recommender_utils.preprocess_data_to_graph(data_array_train, dtypes=recommender_utils.dtypes, class_values=recommender_utils.class_values)
train_item_features = sp.csr_matrix(recommender_utils.get_movies_features(movie_df, train_item_dict, genre_headers, num_genres))
val_adjacency_mx, val_labels, val_user_idx, val_item_idx, val_item_dict = recommender_utils.preprocess_data_to_graph(data_array_val, dtypes=recommender_utils.dtypes, class_values=recommender_utils.class_values)
val_item_features = sp.csr_matrix(recommender_utils.get_movies_features(movie_df, val_item_dict, genre_headers, num_genres))
test_adjacency_mx, test_labels, test_user_idx, test_item_idx, test_item_dict = recommender_utils.preprocess_data_to_graph(data_array_test, dtypes=recommender_utils.dtypes, class_values=recommender_utils.class_values)
test_item_features = sp.csr_matrix(recommender_utils.get_movies_features(movie_df, test_item_dict, genre_headers, num_genres))

print("Train item features shape: "+str(train_item_features.shape))
print("Validation item features shape: "+str(val_item_features.shape))
print("Test item features shape: "+str(test_item_features.shape))


Train item features shape: (8500, 18)
Validation item features shape: (5536, 18)
Test item features shape: (3975, 18)


In [30]:
%autoreload 2
import torch
import math
from torch.optim import Adam
import torch.nn.functional as F

import model_GNN as model_gnn


In [27]:
use_item_features = False
train_item_features_array = train_item_features.toarray() if use_item_features else None

train_dataset = MyDynamicDataset(root='data_test/processed/train', A=train_adjacency_mx, 
    links=(train_user_idx, train_item_idx), labels=train_labels, h=1, sample_ratio=1.0, 
    max_nodes_per_hop=200, u_features=None, v_features=train_item_features_array, class_values=class_values)
test_dataset = MyDataset(root='data_test/processed/test', A=test_adjacency_mx, 
    links=(test_user_idx, test_item_idx), labels=test_labels, h=1, sample_ratio=1.0, 
    max_nodes_per_hop=200, u_features=None, v_features=train_item_features_array, class_values=class_values)

In [28]:
LR = 1e-3
EPOCHS = 10
BATCH_SIZE = 50
LR_DECAY_STEP = 20
LR_DECAY_VALUE = 10

train_loader = DataLoader(train_dataset, BATCH_SIZE, shuffle=True, num_workers=2)
test_loader = DataLoader(test_dataset, BATCH_SIZE, shuffle=False, num_workers=2)

device = torch.device("cpu")
model = model_gnn.IGMC()
model.to(device)
model.reset_parameters()
optimizer = Adam(model.parameters(), lr=LR, weight_decay=0)

/Users/user/Documents/Nauka/Studia/Magisterskie/Magisterka/Recommender/.venv/lib/python3.9/site-packages/torch_geometric/deprecation.py:12: UserWarning: 'data.DataLoader' is deprecated, use 'loader.DataLoader' instead
  warnings.warn(out)


In [29]:
loss_through_epochs = []
batches_per_epoch = len(train_loader)
for epoch in range(1, EPOCHS+1):
    model.train()
    train_loss_all = 0
    for i, train_batch in enumerate(train_loader):
        if i % 100 == 0 or i % batches_per_epoch == 0:
            print(f"{i}/{batches_per_epoch}")
        optimizer.zero_grad()
        train_batch = train_batch.to(device)
        y_pred = model(train_batch)
        y_true = train_batch.y
        train_loss = F.mse_loss(y_pred, y_true)
        train_loss.backward()
        train_loss_all += BATCH_SIZE * float(train_loss)
        optimizer.step()
        torch.cuda.empty_cache()
    train_loss_all = train_loss_all / len(train_loader.dataset)
    loss_through_epochs.append(train_loss_all)
    print('epoch', epoch,'; train loss', train_loss_all)

    if epoch % LR_DECAY_STEP == 0:
      for param_group in optimizer.param_groups:
          param_group['lr'] = param_group['lr'] / LR_DECAY_VALUE

0/1370


/Users/user/Documents/Nauka/Studia/Magisterskie/Magisterka/Recommender/.venv/lib/python3.9/site-packages/torch_geometric/deprecation.py:12: UserWarning: 'dropout_adj' is deprecated, use 'dropout_edge' instead
  warnings.warn(out)


100/1370


KeyboardInterrupt: 

In [72]:
batches_in_eval = len(test_loader)
model.eval()
test_loss = 0
for i, test_batch in enumerate(test_loader):
    print(f"{i}/{batches_in_eval}")
    test_batch = test_batch.to(device)
    with torch.no_grad():
        y_pred = model(test_batch)
    y_true = test_batch.y
    test_loss += F.mse_loss(y_pred, y_true, reduction='sum')
    torch.cuda.empty_cache()
mse_loss = float(test_loss) / len(test_loader.dataset)

print('test MSE loss', mse_loss)
print('test RMSE loss', math.sqrt(mse_loss))

0/99
1/99
2/99
3/99
4/99
5/99
6/99
7/99
8/99
9/99
10/99
11/99
12/99
13/99
14/99
15/99
16/99
17/99
18/99
19/99
20/99
21/99
22/99
23/99
24/99
25/99
26/99
27/99
28/99
29/99
30/99
31/99
32/99
33/99
34/99
35/99
36/99
37/99
38/99
39/99
40/99
41/99
42/99
43/99
44/99
45/99
46/99
47/99
48/99
49/99
50/99
51/99
52/99
53/99
54/99
55/99
56/99
57/99
58/99
59/99
60/99
61/99
62/99
63/99
64/99
65/99
66/99
67/99
68/99
69/99
70/99
71/99
72/99
73/99
74/99
75/99
76/99
77/99
78/99
79/99
80/99
81/99
82/99
83/99
84/99
85/99
86/99
87/99
88/99
89/99
90/99
91/99
92/99
93/99
94/99
95/99
96/99
97/99
98/99
test MSE loss 0.9694746933974229
test RMSE loss 0.9846190600417112


In [89]:
torch.save(model.state_dict(), "models/graph_80epochs_with_features.pt")
epochs_80_with_features_loss = loss_through_epochs